In [1]:
import numpy as np
import pandas as pd
import string

# Read data
train = pd.read_csv(r"D:\data mining\HW4\train.csv", sep = '\t')
test = pd.read_csv(r"D:\data mining\HW4\test.csv", sep = '\t')
label = pd.read_csv(r"D:\data mining\HW4\sample_submission.csv", sep = ',')

In [2]:
# # Data Preprocessing
train.text = train.text.str.lower()
test.text = test.text.str.lower()

In [3]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer

tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+')
porter_stemmer = PorterStemmer()

# train data
for i in range(4987):
    # Tokenize
    train.text[i] = tokenizer.tokenize(train.text[i])
        
    # Stemming
    train.text[i] = " ".join([porter_stemmer.stem(word = word) for word in train.text[i]])
# test data
for i in range(1247):
    # Tokenize
    test.text[i] = tokenizer.tokenize(test.text[i])
        
    # Stemming
    test.text[i] = " ".join([porter_stemmer.stem(word = word) for word in test.text[i]])

D:\anaconda\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\anaconda\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
# Stopwords
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
EngStopWords = set(stopwords.words('english'))#這裡設定稍後取用 English 的停用詞語料庫

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\葉之晴\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Word to TfidfVec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

vectorizer = TfidfVectorizer(stop_words = EngStopWords, token_pattern = "(?u)\\b\\w+\\b", smooth_idf = True, max_features = 10000)
x_train = vectorizer.fit_transform(train.text).toarray()
x_test = vectorizer.fit_transform(test.text).toarray()
y_train = train.label
y_test = label.label.astype(str)

In [6]:
# Evaluation Function
from sklearn import metrics

def Evaluation(y_test, prediction):
    print("Confusion Matrix:")
    print(metrics.confusion_matrix(y_test, prediction))
    print("Accuracy: " , (metrics.accuracy_score(y_test, prediction)))
    print("Precision: " , (metrics.precision_score(y_test, prediction, pos_label = '1')))
    print("Recall: " , (metrics.recall_score(y_test, prediction, pos_label = '1')))
    print("F-measure: " , (metrics.f1_score(y_test, prediction, pos_label = '1')))

In [7]:
# XGBoost
from xgboost import XGBClassifier

XGBoost_model = XGBClassifier(n_estimators = 100, max_features = 100, max_depth = 5, learning_rate = 0.1)
XGBoost_model.fit(x_train, y_train)
preds = XGBoost_model.predict(x_test)
Evaluation(y_test, preds)

D:\anaconda\lib\site-packages\dask\config.py:131: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
D:\anaconda\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


[17:03:51] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { max_features } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Confusion Matrix:
[[549  81]
 [526  91]]
Accuracy:  0.5132317562149158
Precision:  0.5290697674418605
Recall:  0.14748784440842788
F-measure:  0.23067173637515845


In [8]:
# GBDT
from sklearn.ensemble import GradientBoostingClassifier

GBDT_model = GradientBoostingClassifier(n_estimators = 100, max_features = 100, max_depth = 5, learning_rate = 0.1)
GBDT_model.fit(x_train, y_train)
prediction = GBDT_model.predict(x_test)
Evaluation(y_test,prediction)

Confusion Matrix:
[[590  40]
 [581  36]]
Accuracy:  0.5020048115477145
Precision:  0.47368421052631576
Recall:  0.05834683954619125
F-measure:  0.10389610389610389


In [10]:
# LightGBM
from lightgbm import LGBMClassifier

LightGBM_model = LGBMClassifier(n_estimators = 100, max_depth = 5, learning_rate = 0.1)
LightGBM_model.fit(x_train, y_train)
prediction = LightGBM_model.predict(x_test)
Evaluation(y_test,prediction)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Confusion Matrix:
[[572  58]
 [537  80]]
Accuracy:  0.5228548516439455
Precision:  0.5797101449275363
Recall:  0.12965964343598055
F-measure:  0.2119205298013245
